## VGAE applied in a graph form from pairs of frames concatenated to form single input

In [12]:
%reset

### Install packets that we use for the GVAE

In [24]:
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-geometric

In [50]:
# importing packages
import numpy as np
import scipy as sp
from sklearn.cluster import spectral_clustering
from sklearn.feature_extraction import image
import cv2
import matplotlib.pyplot as plt
from skimage.transform import resize
import os
from os.path import join
import networkx as nx
import scipy.sparse
import glob

## 1. Creating the dataset with frames

In [51]:
dir = r'C:\Users\rscaciot22\OneDrive - Oulun yliopisto\Documents\2022\GRAPH_ENCODER\Simulation\cart-pole-main\GraphEncoder'
os.chdir(dir)
!python ./cart_pole_discreet.py

Testing steps: 29 rewards -148.17218662487778: 


C:\Program Files\Python39\lib\site-packages\gym\spaces\box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


## 2. Convert images to graph

In [55]:
# Apply spectral clustering (this step goes much faster if you have pyamg
# installed)
N_REGIONS = 25
beta = 5
eps = 1e-6
p = 0.1

In [56]:
cam = os.path.join(dir, './data/')
latest_subdir = max([os.path.join(cam,d) for d in os.listdir(cam)], key=os.path.getmtime)
print(str(latest_subdir))
folder = latest_subdir
count = 0
steps = 0
maxPixel= 12
imageSize = maxPixel * maxPixel
num_features = imageSize

cam_graph = os.path.join(dir, './graph/')
print(str(cam_graph))
all_folders = os.listdir(cam_graph)
os.chdir(cam_graph)
retval = os.getcwd()
all_folders.sort()
latest = all_folders[-1].replace('S', '')
new = int(latest) + 1
mydir = 'S'+str(new)
print(str(mydir))
os.makedirs(mydir)

# Iterate directory
for path in os.listdir(folder):
    # check if current path is a file
    if os.path.isfile(os.path.join(folder, path)):
        count += 1
print('File count:', count)


C:\Users\rscaciot22\OneDrive - Oulun yliopisto\Documents\2022\GRAPH_ENCODER\Simulation\cart-pole-main\GraphEncoder\./data/S4
C:\Users\rscaciot22\OneDrive - Oulun yliopisto\Documents\2022\GRAPH_ENCODER\Simulation\cart-pole-main\GraphEncoder\./graph/
S4
File count: 29


In [57]:
for f in range(count):
    print(str(steps))
    print(str(f))
    # reading image
    data = 'Frame' + str(steps) + '.png'
    path = join(folder, data)
    print(path)
    def read_image():
        img = cv2.imread(path, 0)
        return img

    cartpole = read_image()
    print ("img_processing.size=",cartpole.size)
    imageSize=maxPixel*maxPixel
    cartpole = resize(cartpole, (maxPixel, maxPixel))
    print ("img_processing.size=",cartpole.size)
    # Downsample the image by a factor of 4
    print(cartpole.shape)
    mask = cartpole.astype(bool)
    cartpole = cartpole.astype(float)
    print(cartpole.shape)

    # Convert the image into a graph with the value of the gradient on the
    # edges.
    import random

    if True:
        graph = image.img_to_graph(cartpole, mask=mask, return_as=np.ndarray)
        arr = image.img_to_graph(cartpole, mask=mask, return_as=np.ndarray)
        graph1 = image.img_to_graph(cartpole)

        G = nx.from_numpy_array(arr)
        G.edges(data=True)

        row = []
        col = []
        data = []

        for i in range(graph.shape[0]):
            for j in range(graph.shape[1]):
                if graph[i][j] != 0:
                    row.append(i)
                    col.append(j)
                    data.append(graph[i][j])

        graph = sp.sparse.coo_matrix((data, (row,col)), shape=graph.shape, dtype = float)
    else:
        graph = image.img_to_graph(cartpole)

    # Take a decreasing function of the gradient: an exponential
    # The smaller beta is, the more independent the segmentation is of the
    # actual image. For beta=1, the segmentation is close to a voronoi
    graph.data = np.exp(-beta * graph.data / cartpole.std()) + eps
    # use the tofile() method
    # and use ',' as a separator
    # as we have to generate a csv file
    arr.tofile(str(mydir) + '\g'+ str(steps)+'.csv', sep = ',')
    # save in graphml
    nx.write_graphml_lxml(G, str(mydir) + '\g'+ str(steps)+'.graphml')
    # save in npz
    scipy.sparse.save_npz(str(mydir) + '\g'+ str(steps)+'.npz', graph)
    steps += 1
f += 1

0
0
C:\Users\rscaciot22\OneDrive - Oulun yliopisto\Documents\2022\GRAPH_ENCODER\Simulation\cart-pole-main\GraphEncoder\./data/S4\Frame0.png
img_processing.size= 240000
img_processing.size= 144
(12, 12)
(12, 12)
1
1
C:\Users\rscaciot22\OneDrive - Oulun yliopisto\Documents\2022\GRAPH_ENCODER\Simulation\cart-pole-main\GraphEncoder\./data/S4\Frame1.png
img_processing.size= 240000
img_processing.size= 144
(12, 12)
(12, 12)
2
2
C:\Users\rscaciot22\OneDrive - Oulun yliopisto\Documents\2022\GRAPH_ENCODER\Simulation\cart-pole-main\GraphEncoder\./data/S4\Frame2.png
img_processing.size= 240000
img_processing.size= 144
(12, 12)
(12, 12)
3
3
C:\Users\rscaciot22\OneDrive - Oulun yliopisto\Documents\2022\GRAPH_ENCODER\Simulation\cart-pole-main\GraphEncoder\./data/S4\Frame3.png
img_processing.size= 240000
img_processing.size= 144
(12, 12)
(12, 12)
4
4
C:\Users\rscaciot22\OneDrive - Oulun yliopisto\Documents\2022\GRAPH_ENCODER\Simulation\cart-pole-main\GraphEncoder\./data/S4\Frame4.png
img_processing.s

## 3. Concatenating graphs to have one input

In [58]:
folder = str(cam_graph) + str(mydir)
f = 0
F = nx.Graph()
print(str(folder))
# Iterate directory
counter= len(glob.glob1(folder,"*.graphml"))
print('File count .graphml:', counter)

cmap = plt.get_cmap('plasma')

C:\Users\rscaciot22\OneDrive - Oulun yliopisto\Documents\2022\GRAPH_ENCODER\Simulation\cart-pole-main\GraphEncoder\./graph/S4
File count .graphml: 29


In [59]:
for f in range(counter):
    # reading graph
    data = 'g' + str(f) + '.graphml'
    path = join(folder, data)
    print(path)
    def read_graph():
        graph = nx.read_graphml(path)
        return graph

    G = read_graph()
    F = nx.compose(F,G)
    # nx.draw(F)

    # pos = nx.spring_layout(F)
    # fig = plt.figure(figsize=(10,10))
    # nx.draw_networkx_edges(F, with_labels=False, node_size=50, pos=pos, alpha=.3)# node_color=colors,

    # for node in F.nodes():
    #     w = plt.pie(
    #         [1]*counter,
    #         center= pos[node],
    #       colors=  cmap.colors([f]),
    #        radius=0.05,
    #    )
f += 1

C:\Users\rscaciot22\OneDrive - Oulun yliopisto\Documents\2022\GRAPH_ENCODER\Simulation\cart-pole-main\GraphEncoder\./graph/S4\g0.graphml
C:\Users\rscaciot22\OneDrive - Oulun yliopisto\Documents\2022\GRAPH_ENCODER\Simulation\cart-pole-main\GraphEncoder\./graph/S4\g1.graphml
C:\Users\rscaciot22\OneDrive - Oulun yliopisto\Documents\2022\GRAPH_ENCODER\Simulation\cart-pole-main\GraphEncoder\./graph/S4\g2.graphml
C:\Users\rscaciot22\OneDrive - Oulun yliopisto\Documents\2022\GRAPH_ENCODER\Simulation\cart-pole-main\GraphEncoder\./graph/S4\g3.graphml
C:\Users\rscaciot22\OneDrive - Oulun yliopisto\Documents\2022\GRAPH_ENCODER\Simulation\cart-pole-main\GraphEncoder\./graph/S4\g4.graphml
C:\Users\rscaciot22\OneDrive - Oulun yliopisto\Documents\2022\GRAPH_ENCODER\Simulation\cart-pole-main\GraphEncoder\./graph/S4\g5.graphml
C:\Users\rscaciot22\OneDrive - Oulun yliopisto\Documents\2022\GRAPH_ENCODER\Simulation\cart-pole-main\GraphEncoder\./graph/S4\g6.graphml
C:\Users\rscaciot22\OneDrive - Oulun ylio

In [60]:
nx.write_graphml_lxml(F, str(mydir) + 'Concat.graphml')

### Information about data
You have the following attributes:

- x the node features, hence it's dimension is number of nodes times feature dimension
- edge_index the edge list
- y the "ground truth"/class labels or in that specific case the classification of the papers. Hence, it's shape is the number of nodes.
- The three masks: train_mask, val_mask, test_mask. If I access them via data.train_mask, it gives me a boolean tensor with the length = number of nodes. This is the "default split" of the dataset. They should be disjoint and if True the respective node is in that set.

In [ ]:
from torch_geometric.utils.convert import from_networkx

In [ ]:
pyg_graph = from_networkx(F)
print(pyg_graph)
print(pyg_graph.x)
print(pyg_graph.y)
print(pyg_graph.edge_index)

In [ ]:
# Split the data
train_ratio = 0.2
num_nodes = pyg_graph.x.shape[0]
num_train = int(num_nodes * train_ratio)
idx = [i for i in range(num_nodes)]

np.random.shuffle(idx)
train_mask = torch.full_like(pyg_graph.y, False, dtype=bool)
train_mask[idx[:num_train]] = True
test_mask = torch.full_like(pyg_graph.y, False, dtype=bool)
test_mask[idx[num_train:]] = True
val_mask = test_mask

print(train_mask)
print(test_mask)
print(val_mask)

## 4. Training the VGAE